In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.python.keras.constraints import maxnorm
from tensorflow.python.keras.utils import np_utils

train_images = pd.read_pickle('D:/Study/Fall2019/COMP551/551A3/train_max_x.zip') 
x_train = np.reshape(train_images, (50000, 128, 128))
x_train = x_train.reshape(x_train.shape[0], 128, 128, 1)

df = pd.read_csv('D:/Study/Fall2019/COMP551/551A3/train_y_mod.csv')
y_train = df.Label.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


# Reshaping the array to 4-dims so that it can work with the Keras API
X_train = X_train.reshape(X_train.shape[0], 128, 128, 1)
X_test = X_test.reshape(X_test.shape[0], 128, 128, 1)
input_shape = (128, 128, 1)
# Making sure that the values are float so that we can get decimal points after division
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
#X_train /= 255.0
#X_test /= 255.0
print('x_train shape:', X_train.shape)
print('Number of images in x_train', X_train.shape[0])
print('Number of images in x_test', X_test.shape[0])

x_train shape: (40000, 128, 128, 1)
Number of images in x_train 40000
Number of images in x_test 10000


In [2]:
X_train_rot = np.copy(X_train)

In [4]:
from tensorflow.keras.preprocessing import image
for i in range(X_train_rot.shape[0]):
    newImage = image.random_rotation(x=X_train[i], rg = 45, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest', cval=0.0,interpolation_order=3)
    X_train_rot[i] = newImage
    

In [5]:
X_train /= 255.0
X_train_rot /= 255.0
X_test /= 255.0

X_train[X_train < 0.8] = 0
X_train[X_train >= 0.8] = 1

X_train_rot[X_train_rot < 0.8] = 0
X_train_rot[X_train_rot >= 0.8] = 1

X_test[X_test < 0.8] = 0
X_test[X_test >= 0.8] = 1

In [6]:
input_shape = (128, 128, 1)

cnn4 = Sequential()
cnn4.add(Conv2D(32, kernel_size=5, strides=1, input_shape=input_shape))
cnn4.add(BatchNormalization())
cnn4.add(Activation('relu'))

cnn4.add(Conv2D(32, kernel_size=5, strides=1))
cnn4.add(BatchNormalization())
cnn4.add(Activation('relu'))
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
#cnn4.add(Dropout(0.2))

cnn4.add(Conv2D(64, kernel_size=5, strides=1,  padding='same'))
cnn4.add(BatchNormalization())
cnn4.add(Activation('relu'))

cnn4.add(Conv2D(64, kernel_size=4, padding='same', strides=1))
cnn4.add(BatchNormalization())
cnn4.add(Activation('relu'))
#cnn4.add(MaxPooling2D())
#cnn4.add(Dropout(0.2))

cnn4.add(Conv2D(128, kernel_size=5, strides=1))
cnn4.add(BatchNormalization())
cnn4.add(Activation('relu'))
#cnn4.add(MaxPooling2D())
#cnn4.add(Dropout(0.2))


cnn4.add(Conv2D(128, kernel_size=5, strides=1))
cnn4.add(BatchNormalization())
cnn4.add(Activation('relu'))
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
#cnn4.add(Dropout(0.2))


cnn4.add(Flatten())

cnn4.add(Dense(1024, activation='relu'))
#cnn4.add(BatchNormalization())
#cnn4.add(Dropout(0.2))

#cnn4.add(Dense(256, activation='relu'))
#cnn4.add(BatchNormalization())
#cnn4.add(Dropout(0.2))

cnn4.add(Dense(10, activation='softmax'))

cnn4.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
for ep in range(10):
    cnn4.fit(x=X_train ,y=y_train, validation_data = (X_test, y_test), epochs=1, batch_size = 32)
    cnn4.fit(x=X_train_rot ,y=y_train, validation_data = (X_test, y_test), epochs=1, batch_size = 32)
    #cnn4.fit(x=X_train2,y=y_train2, epochs=1)
    #cnn4.fit(x=X_train3,y=y_train3, epochs=1)
    #cnn4.fit(x=X_train4,y=y_train4, epochs=1)

Train on 40000 samples, validate on 10000 samples
40000/40000 [==============================] - 117s 3ms/sample - loss: 4.1727 - acc: 0.2465 - val_loss: 1.8564 - val_acc: 0.2875
Train on 40000 samples, validate on 10000 samples
40000/40000 [==============================] - 110s 3ms/sample - loss: 1.7712 - acc: 0.3322 - val_loss: 1.8941 - val_acc: 0.2333
Train on 40000 samples, validate on 10000 samples
40000/40000 [==============================] - 110s 3ms/sample - loss: 1.2976 - acc: 0.5467 - val_loss: 1.9293 - val_acc: 0.5216
Train on 40000 samples, validate on 10000 samples
40000/40000 [==============================] - 110s 3ms/sample - loss: 1.0007 - acc: 0.6586 - val_loss: 0.6965 - val_acc: 0.7626
Train on 40000 samples, validate on 10000 samples
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.5023 - acc: 0.8389 - val_loss: 0.5676 - val_acc: 0.8187
Train on 40000 samples, validate on 10000 samples
40000/40000 [==============================] - 110s 3ms

In [8]:
cnn4.fit(x=X_train ,y=y_train, validation_data = (X_test, y_test), epochs=1, batch_size = 32)
cnn4.fit(x=X_train_rot ,y=y_train, validation_data = (X_test, y_test), epochs=1, batch_size = 32)
cnn4.fit(x=X_train ,y=y_train, validation_data = (X_test, y_test), epochs=1, batch_size = 32)
cnn4.fit(x=X_train_rot ,y=y_train, validation_data = (X_test, y_test), epochs=1, batch_size = 32)

Train on 40000 samples, validate on 10000 samples
40000/40000 [==============================] - 109s 3ms/sample - loss: 0.0288 - acc: 0.9923 - val_loss: 0.4079 - val_acc: 0.9309
Train on 40000 samples, validate on 10000 samples
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0348 - acc: 0.9895 - val_loss: 0.5144 - val_acc: 0.9080
Train on 40000 samples, validate on 10000 samples
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0254 - acc: 0.9926 - val_loss: 0.4119 - val_acc: 0.9309
Train on 40000 samples, validate on 10000 samples
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0325 - acc: 0.9905 - val_loss: 0.5496 - val_acc: 0.9110


In [9]:
y_result = cnn4.predict(x=X_test)

In [17]:
result = []
for i in range(10000):
    result.append(np.argmax(y_result[i]))
#print(y_result[1])
#np.argmax(y_result[1])

In [18]:
print(result)

[8, 9, 9, 6, 6, 4, 6, 8, 9, 9, 9, 9, 7, 7, 7, 2, 6, 5, 5, 6, 9, 9, 8, 7, 7, 9, 7, 9, 9, 4, 8, 5, 5, 5, 8, 9, 9, 4, 7, 4, 8, 6, 8, 5, 7, 6, 7, 9, 8, 6, 6, 8, 7, 8, 9, 8, 5, 7, 9, 9, 8, 5, 8, 5, 5, 6, 2, 3, 7, 2, 9, 9, 9, 9, 7, 5, 6, 9, 7, 8, 3, 7, 7, 5, 4, 7, 7, 9, 9, 9, 9, 9, 4, 4, 5, 9, 9, 9, 8, 9, 5, 6, 7, 8, 4, 7, 6, 4, 6, 6, 9, 8, 7, 8, 8, 9, 3, 7, 7, 9, 7, 6, 6, 3, 8, 9, 7, 7, 8, 8, 9, 6, 9, 9, 8, 4, 4, 6, 9, 8, 8, 8, 8, 9, 7, 8, 8, 9, 3, 9, 3, 6, 7, 7, 6, 8, 6, 7, 6, 6, 7, 5, 3, 8, 9, 8, 3, 8, 9, 5, 6, 7, 8, 8, 8, 9, 8, 8, 6, 4, 7, 7, 5, 8, 7, 8, 7, 8, 8, 9, 7, 7, 8, 9, 3, 9, 9, 3, 4, 8, 6, 6, 9, 8, 7, 5, 7, 9, 5, 9, 4, 9, 7, 9, 7, 8, 8, 8, 8, 9, 7, 8, 3, 8, 5, 8, 4, 7, 5, 9, 7, 7, 8, 8, 7, 9, 8, 9, 8, 8, 9, 9, 9, 9, 7, 7, 5, 9, 3, 9, 5, 9, 3, 6, 5, 9, 7, 7, 4, 7, 7, 7, 9, 7, 9, 7, 6, 3, 5, 9, 9, 4, 7, 9, 9, 8, 9, 9, 3, 8, 8, 9, 9, 4, 9, 9, 9, 6, 7, 5, 7, 9, 9, 6, 7, 7, 3, 5, 8, 9, 4, 7, 8, 7, 9, 3, 9, 8, 9, 9, 8, 4, 9, 9, 6, 5, 9, 3, 7, 6, 7, 7, 6, 5, 9, 9, 6, 8, 8, 7, 8, 7, 8, 

In [6]:
cnn4.fit(x=X_train ,y=y_train, validation_data = (X_test, y_test), epochs=15, batch_size = 32)

Train on 40000 samples, validate on 10000 samples
Epoch 1/15
40000/40000 [==============================] - 111s 3ms/sample - loss: 0.0605 - acc: 0.9821 - val_loss: 0.3538 - val_acc: 0.9281
Epoch 2/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0489 - acc: 0.9852 - val_loss: 0.4163 - val_acc: 0.9214
Epoch 3/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0413 - acc: 0.9876 - val_loss: 0.4544 - val_acc: 0.9199
Epoch 4/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0357 - acc: 0.9897 - val_loss: 0.4952 - val_acc: 0.9152
Epoch 5/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0344 - acc: 0.9902 - val_loss: 0.4842 - val_acc: 0.9215
Epoch 6/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0331 - acc: 0.9910 - val_loss: 0.4213 - val_acc: 0.9229
Epoch 7/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0319 - acc: 0.9909 - val_l

In [7]:
cnn4.fit(x=X_train ,y=y_train, validation_data = (X_test, y_test), epochs=15, batch_size = 32)

Train on 40000 samples, validate on 10000 samples
Epoch 1/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0187 - acc: 0.9950 - val_loss: 0.5996 - val_acc: 0.9174
Epoch 2/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0133 - acc: 0.9965 - val_loss: 0.5581 - val_acc: 0.9224
Epoch 3/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0183 - acc: 0.9954 - val_loss: 0.5028 - val_acc: 0.9261
Epoch 4/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0126 - acc: 0.9970 - val_loss: 0.4953 - val_acc: 0.9258
Epoch 5/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0124 - acc: 0.9973 - val_loss: 0.6571 - val_acc: 0.9048
Epoch 6/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0126 - acc: 0.9963 - val_loss: 0.6134 - val_acc: 0.9176
Epoch 7/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0166 - acc: 0.9959 - val_l

In [8]:
cnn4.fit(x=X_train ,y=y_train, validation_data = (X_test, y_test), epochs=15, batch_size = 32)

Train on 40000 samples, validate on 10000 samples
Epoch 1/15
40000/40000 [==============================] - 109s 3ms/sample - loss: 0.0122 - acc: 0.9970 - val_loss: 0.6562 - val_acc: 0.9229
Epoch 2/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0125 - acc: 0.9971 - val_loss: 0.6112 - val_acc: 0.9204
Epoch 3/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0039 - acc: 0.9990 - val_loss: 0.6860 - val_acc: 0.9137
Epoch 4/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0085 - acc: 0.9976 - val_loss: 0.6985 - val_acc: 0.9118
Epoch 5/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0071 - acc: 0.9981 - val_loss: 0.6678 - val_acc: 0.9237
Epoch 6/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0063 - acc: 0.9985 - val_loss: 0.6328 - val_acc: 0.9073
Epoch 7/15
40000/40000 [==============================] - 110s 3ms/sample - loss: 0.0110 - acc: 0.9973 - val_l

In [6]:
test_loss, test_acc = cnn4.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

10000/10000 - 9s - loss: 0.9685 - acc: 0.7185

Test accuracy: 0.7185


In [13]:
test_loss, test_acc = cnn4.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

10000/10000 - 10s - loss: 0.9984 - acc: 0.7238

Test accuracy: 0.7238


In [7]:
test_loss, test_acc = cnn4.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

10000/10000 - 14s - loss: 0.9785 - acc: 0.7281

Test accuracy: 0.7281


In [19]:
test_loss, test_acc = cnn4.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

10000/10000 - 12s - loss: 0.3689 - acc: 0.8904

Test accuracy: 0.8904


In [24]:
cnn4.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 124, 124, 32)      832       
_________________________________________________________________
batch_normalization_48 (Batc (None, 124, 124, 32)      128       
_________________________________________________________________
activation_48 (Activation)   (None, 124, 124, 32)      0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 120, 120, 32)      25632     
_________________________________________________________________
batch_normalization_49 (Batc (None, 120, 120, 32)      128       
_________________________________________________________________
activation_49 (Activation)   (None, 120, 120, 32)      0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 60, 60, 32)       

In [34]:
import csv
with open('D:/Study/Fall2019/COMP551/551A3/submit.txt', mode='w') as writingFile:
    writer = csv.writer(writingFile)
    writer.writerow(['id', 'Label'])
    for index in range(10000):
        writer.writerow([index, result[index]])

In [ ]:
csv_writer = csv.writer(f)
 
csv_writer.writerow(header) # write header
 
for row in rows:
    csv_writer.writerow(row)

In [26]:
with open('D:/Study/Fall2019/COMP551/551A3/submit.csv', mode='w') as employee_file:
    employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    employee_writer.writerow(['John Smith', 'Accounting', 'November'])
    employee_writer.writerow(['Erica Meyers', 'IT', 'March'])